In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

from cStringIO import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

import pandas as pd
import numpy as np
import re
# import pycountry
from textblob import TextBlob
from calendar import month_name
from collections import Counter
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')


In [3]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [4]:
import glob

text_lst = []
for filename in glob.glob('*.pdf'):
    try: 
        text_lst.append(convert_pdf_to_txt(filename))
        print("Successful read file: ", filename)
    except:
        print("Can't read file: ", filename)

In [5]:
len(text_lst)

7

In [6]:
text_lst[1]

'A\n\nC\n\nI\n\nN\n\nA\nCE\nO\nL\nA\nN\nO\n\nI\n\nT\nA\n\nN\n\nU\n\n.\n\nS\n\n.\n\nD\n\nE\n\nP\n\nA\n\nTNEMTR\n\nN D A T OSPHE\n\nM\n\nRIC\n\nA\n\nD\n\nM\n\nI\n\nN\n\nI\n\nS\n\nT\nR\nA\nT\nI\nO\nN\nE\n\nM E RC\n\nM\n\nO F C O\n\nOil and Hazardous Materials\nResponse Reports\n\nOctober 1993-September 1994\n\nFebruary 1995\n\nNational Oceanic and Atmospheric Administration\nOffice of Ocean Resources Conservation and Assessment\nHazardous Materials Response and Assessment  Division\nSeattle, Washington  98115\n\n\x0c\x0cContents\n\n     Page\n\nIntroduction .......................................... i\n\nSpill Report Keys ................................ ii\n\nFY94 Spill Responses......................... ix\n\nFY94 Simulation Drills ...................xvii\n\nDistrict 1 ............................................... 1\n\nDistrict 2/9......................................... 21\n\nDistrict 5 ............................................. 35\n\nDistrict 7 ..................................

In [7]:
complete_info = re.sub(r'\n', "@", text_lst[0])
complete_info = re.findall(r'USCG District \d+.*?Incident Summary\:.*?\@\@.*?\@\@.*?\@\@.*?\@\@', complete_info)
# complete_info = re.findall(r'Incident Summary\:.*\@\@', complete_info)

In [8]:
len(complete_info)

45

In [9]:
a = complete_info[:5]
a

["USCG District 1@@2@@\x0cName of Spill: @NOAA SSC: @USCG District:@Date of Spill: @Location of Spill: @Latitude: @Longitude: @Spilled Material: @Spilled Material Type: @Amount:@@Source of Spill: @Resources at Risk: @@Dispersants: @Bioremediation: @In-situ Burning:@Other Special Interest: @Shoreline Types Impacted:@Keywords: @@Incident Summary:@@USCG District 1@@MSC Chiara@Stephen Lehmann@1@12/11/92@Buzzards Bay, Massachusetts@41\xc2\xb038.6\xe2\x80\x99 N@070\xc2\xb041.4' W@Bunker C and diesel@4 and 2@5,300 barrels Bunker C@1,500 barrels diesel@container vessel@wintering ducks, lobster, quohogs, clams, scallops,@oysters, and sheltered tidal flats@N@N@N@dangerous cargo, extreme weather conditions@none@none@@At approximately 2030, December 10, 1992, the MSC Chiara, a Panamanian-flagged, Swiss-@owned container vessel, struck a submerged object after exiting the Cape Cod Canal into@Buzzards Bay, Massachusetts.  The vessel\xe2\x80\x99s Master, fearing for the stability of his vessel, put@he

In [15]:
print "ABC\n"

In [317]:
text0 = complete_info[44].replace('\xc2\xb0'," degree ").replace('\xe2\x80\x99','').replace("\x0c","").split("@")
text0 = "@".join(text0)
text0

'USCG District 17@@125@@USCG District 17@@Name of Spill:   @NOAA SSC:   @USCG District:@Date of Spill:   @Location of Spill:   @Latitude:   @Longitude:   @Spilled Material:   @Spilled Material Type: @Quantity:    @Source of Release:   @Resources at Risk: @Dispersants:@Bioremediation:@In-situ Burning:@Other Special Interest: @Keywords: @@Incident Summary:@@M/V Sun Tide@John W. Whitney@17@08/23/93@Upper Cook Inlet, Alaska@61 degree 00 N@151 degree 00 W@diesel@2@6,000 gallons@non-tank vessel@waterfowl@N@N@N@none@evaporation@@At 0300 on August 23, 1993, the spill response vessel, M/V Sun Tide collided with the ARCO@jack-up drilling rig, Gilbert Rowe, and ruptured a diesel fuel tank releasing 6,000 gallons into@Upper Cook Inlet between the North Forelands and Possession Point.  The first overflight at@daylight reported a one- by two-mile rainbow sheen.  Subsequent overflights showed the@product to be dispersing and evaporating rapidly and by early afternoon the sheen had@nearly disappeared.

## Extract metric values

In [318]:
info_need = re.findall(r'(?<=Incident Summary:@@)[A-Z].*@@',text0)
info_need

['M/V Sun Tide@John W. Whitney@17@08/23/93@Upper Cook Inlet, Alaska@61 degree 00 N@151 degree 00 W@diesel@2@6,000 gallons@non-tank vessel@waterfowl@N@N@N@none@evaporation@@At 0300 on August 23, 1993, the spill response vessel, M/V Sun Tide collided with the ARCO@jack-up drilling rig, Gilbert Rowe, and ruptured a diesel fuel tank releasing 6,000 gallons into@Upper Cook Inlet between the North Forelands and Possession Point.  The first overflight at@daylight reported a one- by two-mile rainbow sheen.  Subsequent overflights showed the@product to be dispersing and evaporating rapidly and by early afternoon the sheen had@nearly disappeared.  The vessel laid out its own boom and CISPRI deployed 18 response@vessels, including  nine fishing boats to tow boom.   Weather was mild with light winds and@a two-foot chop on the water.@@Behavior of Oil:@@']

In [193]:
metric_values = re.findall(r'^.*?@@', info_need[0])
metric_values = re.sub(r'(?<=@@)\w+$',"", metric_values[0])
# metric_values = re.findall(r'(?<=@@).*', metric_values)
metric_values = metric_values.split('@')
metric_values = [i.lower() for i in metric_values if i!='']
metric_values

['pirate well platform',
 'mike barnhill',
 '8',
 '10/30/92',
 'mississippi river delta, louisiana',
 '28 degree 55 n',
 '89 degree 57 w',
 'unknown',
 'unknown',
 'unknown',
 'undetermined',
 'birds, fish, shellfish, marsh',
 'n',
 'n',
 'n',
 'none',
 'undetermined',
 'none']

In [266]:
latitude = re.findall(r'\d+\s*degree\s*[\d]+\s*N?(?=\@)', info_need[0])
print latitude

In [124]:
accident_desc = re.findall(r' [A-Z][a-z]+ \d+[\s,].*', info_need[0])
accident_desc

[' August 23, 1993, the spill response vessel, M/V Sun Tide collided with the ARCO@jack-up drilling rig, Gilbert Rowe, and ruptured a diesel fuel tank releasing 6,000 gallons into@Upper Cook Inlet between the North Forelands and Possession Point.  The first overflight at@daylight reported a one- by two-mile rainbow sheen.  Subsequent overflights showed the@product to be dispersing and evaporating rapidly and by early afternoon the sheen had@nearly disappeared.  The vessel laid out its own boom and CISPRI deployed 18 response@vessels, including  nine fishing boats to tow boom.   Weather was mild with light winds and@a two-foot chop on the water.@@']

## Extract metric names

In [101]:
# metrics = re.sub(r'USCG',"",text0.replace("  "," "))
# metrics = re.sub(r'\s',"_",metrics)
# target = "Name of Spill\s*(?=\:)|NOAA SSC\s*(?=\:)|USCG District\s*(?=\:)|Date of Spill\s*(?=\:)|Location of Spill\s*(?=\:)|Latitude\s*(?=\:)|Longitude\s*(?=\:)|Spilled Material\s*(?=\:)|Spilled Material Type\s*(?=\:)|Amount\s*(?=\:)|Source of Spill\s*(?=\:)|Resources at Risk\s*(?=\:)|Dispersants\s*(?=\:)|Bioremediation\s*(?=\:)|In-situ Burning\s*(?=\:)|Other Special Interest\s*(?=\:)|Shoreline Types Impacted\s*(?=\:)|Keywords\s*(?=\:)"
metrics = re.findall(r'.*(?=\@\@Incident Summary\:)', text0)
target = "\@[A-Z][^\d]+\s*?\:"
metrics = re.findall(target, metrics[0])
# metrics = re.findall(r'[A-Z].*\s*(?=\:)',metrics)
# metrics = re.sub(r'\d+','',metrics[0])
# # metrics = re.sub(r'\s@|@@|@(?=[A-Z])','',metrics)
# metrics = metrics[0].split(':')
# metrics = [re.sub(r'\s@|@@|@(?=[A-Z])','',i).lower() for i in metrics if i!='']
metrics

['@References:',
 '@Washington, D.C.:',
 '@Name of Spill:  @NOAA SSC:  @USCG District:@Date of Spill :  @Location of Spill: @Spilled Material:  @Spilled Material Type: @Amount:  @Source of Spill: @Resources at Risk: @Dispersants: @Bioremediation: @In-situ Burning: @Other Special Interest: @Shoreline Types Impacted:  @Keywords:']

In [85]:
metric_values

['m/v sun tide',
 'john w. whitney',
 '17',
 '08/23/93',
 'upper cook inlet, alaska',
 '61 degree 00 n',
 '151 degree 00 w',
 'diesel',
 '2',
 '6,000 gallons',
 'non-tank vessel',
 'waterfowl',
 'n',
 'n',
 'n',
 'none',
 'evaporation']

In [22]:
df = pd.DataFrame(data=[metric_values], columns=metrics)

In [23]:
df

Name of Spill: NOAA SSC: USCG District: Date of Spill :  \
0  mystery spill off virginia beach  gary ott              5        10/13/93   

         Location of Spill: Spilled Material: Spilled Material Type:  Amount:  \
0  virginia beach, virginia   heavy black oil                      4  unknown   

  Source of Spill: Resources at Risk: Dispersants: Bioremediation:  \
0   unknown vessel               none            n               n   

  In-situ Burning: Other Special Interest: Shoreline Types Impacted: Keywords:  
0                n                    none                      none      none

In [469]:
len(metrics)

15

In [470]:
len(metric_values)

16

In [70]:
dfs = []
target = "Name of Spill\s*(?=\:|\s*)|NOAA SSC\s*(?=\:|\s*)|USCG District\s*(?=\:)|Date of Spill\s*(?=\:|\s*)|Location of Spill\s*(?=\:|\s*)|Latitude\s*(?=\:|\s*)|Longitude\s*(?=\:|\s*)|Spilled Material\s*(?=\:)|Spilled Material Type\s*(?=\:|\s*)|Spilled Material\s*(?=\s)|Quantity\s*(?=\:|\s*)|Source of Release\s*(?=\:|\s*)|Amount\s*(?=\:|\s*)|Source of Spill\s*(?=\:|\s*)|Resources at Risk\s*(?=\:|\s*)|Dispersants\s*(?=\:|\s*)|Bioremediation\s*(?=\:|\s*)|In-situ Burning\s*(?=\:|\s*)|Other Special Interest\s*(?=\:|\s*)|Shoreline Types Impacted\s*(?=\:|\s*)|Other Special Interest\s*(?=\:|\s*)|Keywords\s*(?=\:|\s*)"
# for textfile in text_lst[0]:
complete_info = re.sub(r'\n', "@", text_lst[0])
complete_info = re.findall(r'USCG District \d+.*?Incident Summary(?=\:|\s*).*?\@\@.*?\@\@.*?\@\@.*?\@\@', complete_info)
complete_info


["USCG District 1@@2@@\x0cName of Spill: @NOAA SSC: @USCG District:@Date of Spill: @Location of Spill: @Latitude: @Longitude: @Spilled Material: @Spilled Material Type: @Amount:@@Source of Spill: @Resources at Risk: @@Dispersants: @Bioremediation: @In-situ Burning:@Other Special Interest: @Shoreline Types Impacted:@Keywords: @@Incident Summary:@@USCG District 1@@MSC Chiara@Stephen Lehmann@1@12/11/92@Buzzards Bay, Massachusetts@41\xc2\xb038.6\xe2\x80\x99 N@070\xc2\xb041.4' W@Bunker C and diesel@4 and 2@5,300 barrels Bunker C@1,500 barrels diesel@container vessel@wintering ducks, lobster, quohogs, clams, scallops,@oysters, and sheltered tidal flats@N@N@N@dangerous cargo, extreme weather conditions@none@none@@At approximately 2030, December 10, 1992, the MSC Chiara, a Panamanian-flagged, Swiss-@owned container vessel, struck a submerged object after exiting the Cape Cod Canal into@Buzzards Bay, Massachusetts.  The vessel\xe2\x80\x99s Master, fearing for the stability of his vessel, put@he

In [71]:
for i,info in enumerate(complete_info):
    print "\n\n"
#     try:
    text0 = info.replace('\xc2\xb0'," degree ").replace('\xe2\x80\x99','').replace("\x0c","").split("@")
    text0 = "@".join(text0)

    info_need = re.findall(r'(?<=Incident Summary:@@)[A-Z].*\.@@',text0)

    #extract accident description
    accident_desc = re.findall(r' [A-Z][a-z]+ \d+[\s,].*', str(info_need))

    #extract metric values
    metric_values = re.findall(r'^.*?@@', str(info_need))
    metric_values = re.sub(r'(?<=@@)\w+$',"", str(metric_values))
    # metric_values = re.findall(r'(?<=@@).*', metric_values)
    metric_values = metric_values.replace(']',"").replace('[',"").replace("'","").replace('"',"").split('@')
    metric_values = [j.lower() for j in metric_values if j != '']
#         metric_values.append(accident_desc)
#         print metric_values

    #extract metric names
    metrics = re.findall(target, text0)
#         metrics.append('Text')
    for x in range(len(metric_values)):
        if x <= len(metrics)-1:
            print(metrics[x])
        print "\t\t\t", metric_values[x]
        
#         print(len(metric_values), len(metrics))
        #create dataframe for each bonding
#     df = pd.DataFrame(data=[metric_values], columns=metrics)
#     dfs.append(df)
#     except:
#         print("Text file:", i)

In [56]:
len(dfs)

0

In [325]:
def extract_data(text_lst):
    texts = []
    dates = []
    states = []
    losses = []
    causes = []
    owners = []
    deaths = []
    injuries = []
    latitudes = []
    longitudes = []
    for j, textfile in enumerate(text_lst):
        complete_info = re.sub(r'\n', "@", textfile)
        complete_info = re.findall(r'USCG District \d+.*?Incident Summary\:.*?\@\@.*?\@\@.*?\@\@.*?\@\@', complete_info)
        for i, info in enumerate(complete_info):
            try:
                #get accident description
                text0 = info.replace('\xc2\xb0'," degree ").replace('\xe2\x80\x99','').replace("\x0c","").split("@")
                text0 = "@".join(text0)

                info_need = re.findall(r'(?<=Incident Summary:@@)[A-Z].*@@',text0)
                accident_detail = re.findall(r' [A-Z][a-z]+ \d+[\s,].*', str(info_need))

                #get date time
                date_time = re.findall(r'[A-Z].*,\s+\d{4}', str(accident_detail))      

                #get State
                state = re.findall(r'([A-Z]\w+\s*,\s*[A-Z]\w+\s*\w*(?=\.|,|\@))', str(info_need))

                #get loss
                loss = re.findall(r'[,\.\d]+\d+\s*(?:barrels|gallons|pounds)\s*(?:@|\s*)', str(info_need))
                print loss, j, i

                #get cause of loss
                cause = re.findall(r'[Rr]uptur|[Ll]eak|pipe failure|[Ee]xplod|fire|puncture|[Ss]pill|material defect',str(text0))
            #     print cause

                #get owner/operator
                owner = re.findall(r'^.*?(?=\@)', str(info_need))
            #     print owner
            
                #get latitude
                latitude = re.findall(r'\d+[\.\d]+\s*degree\s*\d+[\.\d]+\s*N?(?=\@)',str(info_need))
#                 print latitude
                
                #get longitude
                longitude = re.findall(r'\d+[\.\d]+\s*degree\s*\d+[\.\d]+\s*W?(?=\@)',str(info_need))
#                 print longitude

                if accident_detail != []:
                    texts.append(str(accident_detail))
                    dates.append(date_time)
                    states.append(state)
                    losses.append(loss)
                    causes.append(cause)
                    owners.append(owner)
                    latitudes.append(latitude)
                    longitudes.append(longitude)
            #         deaths.append(death)
            #         injuries.append(injury)
            except:
                print("Problem at file:", i)


        
    return texts, dates, states, losses, causes, owners, latitudes, longitudes
    
            

In [326]:
data = extract_data(text_lst)

In [327]:
len(data)

8

## Framing data

In [348]:
df = pd.DataFrame({'Text': data[0],                                      
                    'Loss Estimated (Gallons|Barrels|Pounds)': data[3],
                    'Name of Operator/Owner': data[5],
                    'Cause of Loss': data[4],                  
                    'State': data[2],
                    'Longitude': data[7],
                    'Latitude': data[6],
                    'Incidence Date': data[1],})

In [349]:
df.insert(1, 'Country', value='USA')
df.insert(len(df.columns), 'Link', value='https://response.restoration.noaa.gov/oil-and-chemical-spills/oil-spills/resources/oil-and-hazardous-materials-response-reports.html')
column_names = [i for i in df.columns.values]

In [354]:
df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: Counter(str(x).lower().split(',')).most_common(1)[0][0])
df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: re.sub(r'spill', "Leak and Spill", str(x)))

In [364]:
for column in column_names:
    df[column] = df[column].apply(lambda x: str(x).replace("[","").replace("]","").replace('\'',"")
                                  .replace('"',"").replace("\\","").replace('(',""))
    df[column] = df[column].apply(lambda x: x='UNKNOWN' if str(x) == '')

SyntaxError: invalid syntax (<ipython-input-364-3c37e47ebac7>, line 4)

In [358]:
df.fil('UNKNOWN')

Cause of Loss Country  \
0     Leak and Spill     USA   
1     Leak and Spill     USA   
2     Leak and Spill     USA   
3     Leak and Spill     USA   
4     Leak and Spill     USA   
5     Leak and Spill     USA   
6     Leak and Spill     USA   
7     Leak and Spill     USA   
8     Leak and Spill     USA   
9     Leak and Spill     USA   
10    Leak and Spill     USA   
11    Leak and Spill     USA   
12    Leak and Spill     USA   
13    Leak and Spill     USA   
14    Leak and Spill     USA   
15    Leak and Spill     USA   
16    Leak and Spill     USA   
17    Leak and Spill     USA   
18    Leak and Spill     USA   
19    Leak and Spill     USA   
20    Leak and Spill     USA   
21    Leak and Spill     USA   
22    Leak and Spill     USA   
23    Leak and Spill     USA   
24    Leak and Spill     USA   
25    Leak and Spill     USA   
26    Leak and Spill     USA   
27    Leak and Spill     USA   
28    Leak and Spill     USA   
29    Leak and Spill     USA   
..               ...     ...   
284   Leak and Spill     USA   
285   Leak and Spill     USA   
286   Leak and Spill     USA   
287   Leak and Spill     USA   
288   Leak and Spill     USA   
289   Leak and Spill     USA   
290   Leak and Spill     USA   
291   Leak and Spill     USA   
292   Leak and Spill     USA   
293   Leak and Spill     USA   
294   Leak and Spill     USA   
295   Leak and Spill     USA   
296   Leak and Spill     USA   
297   Leak and Spill     USA   
298   Leak and Spill     USA   
299   Leak and Spill     USA   
300   Leak and Spill     USA   
301   Leak and Spill     USA   
302   Leak and Spill     USA   
303   Leak and Spill     USA   
304   Leak and Spill     USA   
305   Leak and Spill     USA   
306   Leak and Spill     USA   
307   Leak and Spill     USA   
308   Leak and Spill     USA   
309   Leak and Spill     USA   
310   Leak and Spill     USA   
311   Leak and Spill     USA   
312   Leak and Spill     USA   
313   Leak and Spill     USA   

                                        Incidence Date          Latitude  \
0                                    December 10, 1992  41 degree 38.6 N   
1                                    December 21, 1992  41 degree 18.2 N   
2                                 March 27 or 28, 1993                     
3                                       April 10, 1993                     
4                                         May 25, 1993    40 degree 59 N   
5                                      October 1, 1992    36 degree 09 N   
6                                                       42 degree 23.0 N   
7                                     October 13, 1993                     
8                                                        037 degree 27 N   
9                                     January 26, 1993  034 degree  09 N   
10   January 29, 1993.  The vessel was@carrying abo...    36 degree 58 N   
11                                      March 31, 1993    40 degree 00 N   
12                                                        39 degree 10 N   
13   June 18, 1993, a mystery oil spill was reporte...    36 degree 58 N   
14                                       June 21, 1993  39 degree 56.5 N   
15                                     August 11, 1993                     
16                                   September 1, 1993    35 degree 25 N   
17                                  September 18, 1993    39 degree 30 N   
18                                     January 2, 1993     30 degree 41N   
19                                      March 13, 1993  24 degree 28.9 N   
20                                        May 19, 1993    30 degree 41 N   
21                                        June 8, 1993                     
22                                     August 10, 1993    27 degree 36 N   
23                                    October 30, 1992    28 degree 55 N   
24   December 26, 1992, an ARCO oil rig blew out a ...     29 degree 03N   
25   April 9, 1993, a barge pushed by the tug 

In [356]:
df

Cause of Loss Country  \
0     Leak and Spill     USA   
1     Leak and Spill     USA   
2     Leak and Spill     USA   
3     Leak and Spill     USA   
4     Leak and Spill     USA   
5     Leak and Spill     USA   
6     Leak and Spill     USA   
7     Leak and Spill     USA   
8     Leak and Spill     USA   
9     Leak and Spill     USA   
10    Leak and Spill     USA   
11    Leak and Spill     USA   
12    Leak and Spill     USA   
13    Leak and Spill     USA   
14    Leak and Spill     USA   
15    Leak and Spill     USA   
16    Leak and Spill     USA   
17    Leak and Spill     USA   
18    Leak and Spill     USA   
19    Leak and Spill     USA   
20    Leak and Spill     USA   
21    Leak and Spill     USA   
22    Leak and Spill     USA   
23    Leak and Spill     USA   
24    Leak and Spill     USA   
25    Leak and Spill     USA   
26    Leak and Spill     USA   
27    Leak and Spill     USA   
28    Leak and Spill     USA   
29    Leak and Spill     USA   
..               ...     ...   
284   Leak and Spill     USA   
285   Leak and Spill     USA   
286   Leak and Spill     USA   
287   Leak and Spill     USA   
288   Leak and Spill     USA   
289   Leak and Spill     USA   
290   Leak and Spill     USA   
291   Leak and Spill     USA   
292   Leak and Spill     USA   
293   Leak and Spill     USA   
294   Leak and Spill     USA   
295   Leak and Spill     USA   
296   Leak and Spill     USA   
297   Leak and Spill     USA   
298   Leak and Spill     USA   
299   Leak and Spill     USA   
300   Leak and Spill     USA   
301   Leak and Spill     USA   
302   Leak and Spill     USA   
303   Leak and Spill     USA   
304   Leak and Spill     USA   
305   Leak and Spill     USA   
306   Leak and Spill     USA   
307   Leak and Spill     USA   
308   Leak and Spill     USA   
309   Leak and Spill     USA   
310   Leak and Spill     USA   
311   Leak and Spill     USA   
312   Leak and Spill     USA   
313   Leak and Spill     USA   

                                        Incidence Date          Latitude  \
0                                    December 10, 1992  41 degree 38.6 N   
1                                    December 21, 1992  41 degree 18.2 N   
2                                 March 27 or 28, 1993                     
3                                       April 10, 1993                     
4                                         May 25, 1993    40 degree 59 N   
5                                      October 1, 1992    36 degree 09 N   
6                                                       42 degree 23.0 N   
7                                     October 13, 1993                     
8                                                        037 degree 27 N   
9                                     January 26, 1993  034 degree  09 N   
10   January 29, 1993.  The vessel was@carrying abo...    36 degree 58 N   
11                                      March 31, 1993    40 degree 00 N   
12                                                        39 degree 10 N   
13   June 18, 1993, a mystery oil spill was reporte...    36 degree 58 N   
14                                       June 21, 1993  39 degree 56.5 N   
15                                     August 11, 1993                     
16                                   September 1, 1993    35 degree 25 N   
17                                  September 18, 1993    39 degree 30 N   
18                                     January 2, 1993     30 degree 41N   
19                                      March 13, 1993  24 degree 28.9 N   
20                                        May 19, 1993    30 degree 41 N   
21                                        June 8, 1993                     
22                                     August 10, 1993    27 degree 36 N   
23                                    October 30, 1992    28 degree 55 N   
24   December 26, 1992, an ARCO oil rig blew out a ...     29 degree 03N   
25   April 9, 1993, a barge pushed by the tug 

---


In [357]:
writer = pd.ExcelWriter('response_reports_93_to_99.xlsx')
df.to_excel(writer,'response_reports_93to99')
writer.save()